In [1]:
import requests # to get the HTML
from bs4 import BeautifulSoup # to get parse the text
import numpy as np
import pandas as pd
import tqdm
from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")
# chrome_options.headless = True # also works
driver = webdriver.Chrome(options=chrome_options)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
pd.options.display.max_colwidth = 200

In [11]:
base_url = 'https://www.jobs.bg/'
url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
#      https://www.jobs.bg/front_job_search.php?frompage=15&add_sh=1&from_hp=1&term=#paging

r = driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(html, 'html.parser')

In [12]:
def get_pages(soup):
    """Get the total number of job offerings and the total number of pages.
    output: total number of pages
    """
    total_job_number = soup.find('td',
                            attrs={'style':
                                  'height:25px;width:220px;font-weight:500;padding-bottom:5px;'}).text.split()[-1]
    total_job_number= int(total_job_number)
    total_pages = int(total_job_number/15)+1 #round up
    return total_pages

In [14]:
# total_pages = get_pages(soup)
total_pages = 30
results =[]

#for page in tqdm.tqdm(range(total_pages)):
for page in tqdm.tqdm(range(0, 1)):

    url = f'https://www.jobs.bg/front_job_search.php?frompage={page*15}&add_sh=1&from_hp=1&term=#paging'
    r = driver.get(url)
    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')

    job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})
    
    for i in range(0, 45, 3):
        text_info_elem = job_infos[i]
        job_logo_elem = job_infos[i+1]
        job_company_elem = job_infos[i+2]
        
        position_id = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href'].lstrip('job/')
        position_url = base_url+text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'})['href']
        company_name = job_company_elem.text.strip()

        
        position_r = driver.get(position_url)
        html = driver.execute_script("return document.documentElement.outerHTML")
        position_soup = BeautifulSoup(html, 'html.parser')
        
        position_views = position_soup.find('span', string = 'Разглеждания:').parent.text
        
        iframe_elem = position_soup.find('iframe', attrs = {'id':'customJobIframe'})
        raw_html_elem = position_soup.find('td', attrs = {'class':'jobTitle'})
        if iframe_elem:
            iframe_url = iframe_elem.attrs['src']
            iframe_r = requests.get(iframe_url)
            iframe_soup = BeautifulSoup(iframe_r.text, 'html.parser')
            job_text = iframe_soup.text.strip()
        elif raw_html_elem: 
            job_text=raw_html_elem.parent.parent.text.strip()
        else:
            job_text = np.nan
        
        job_date = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'}).text.strip()
        if job_date == 'днес':
            job_date = date.today().strftime("%d.%m.%y")
        elif job_date == 'вчера':
            job_date = (date.today() - timedelta(days=1)).strftime("%d.%m.%y")

        rating_elem = text_info_elem.find('span',attrs={'class':'iconed'})

        if rating_elem:
            rataing_elem_styles = rating_elem["style"].split(";")
            rating_elem_color_styles = [x for x in rataing_elem_styles if "color:" in x]
            rating_color = rating_elem_color_styles[0].split(":")[1]
            rating_full = rating_elem.text.strip().count("\uf4b3")
            rating_half = (rating_elem.text.strip().count("\uf4b1"))/2
            rating_final = rating_full + rating_half
        else:
            rating_color = np.nan
            rating = np.nan
       
        position = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'}).text.strip()
        benefits = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--body2 top-margin'}).text.strip()
        
        tech_list = []
        for a in text_info_elem.find_all('img'):
            tech = a.attrs['alt']
            tech_list.append(tech)
        tech_list = ", ".join(tech_list)
        results.append([position_id,position_url,company_name,position_views,job_text,job_date, rating_final, rating_color, position, benefits, tech_list])

100%|██████████| 1/1 [00:04<00:00,  4.29s/it]


In [15]:
jobs_df = pd.DataFrame(results, columns=["Position_Id","Position_URL","Company Name","Position Views","Job Description","Date Posted", 
                                         "Position_Rating","Rating Color", "Position", "Benefits", "Technologies"])
jobs_df

,Position_Id,Position_URL,Company Name,Position Views,Job Description,Date Posted,Position_Rating,Rating Color,Position,Benefits,Technologies
0,5657665,https://www.jobs.bg/job/5657665,Домашна Кухня ЕООД,Разглеждания: 984,Продавач-консултант на кулинарен щанд \n\n\n\n\n\t\t\t\t\t\t \t\t\t\t\t\t \tИзисквания:\n\n\nКулинарен щанд „К-Експрес”ф-ма Домашна Кухня ЕООД намиращ се в ХИПЕРМАРКЕТИ...,08.05.21,5.0,#66c1ff,Продавач-консултант на кулинарен щанд,София; Заплата от 900 до 1200 BGN (Нето),
1,5650141,https://www.jobs.bg/job/5650141,Фили Вайб АД,Разглеждания: 4 770,"Описание и изисквания : ""Pizza Lab"" е сред лидерите за най-бързо приготвяне на пици в България. Pizza Lab очаква своите бъдещи колеги на ул. ""Самара 4"", ж.к. Младост 4, новата сградата на Advance ...",07.05.21,5.0,#66c1ff,Асистент в кухня,София,
2,5670852,https://www.jobs.bg/job/5670852,БАКБОУН (БГ) ЕООД,Разглеждания: 1 363,"IT support Technician (Night Shift) Office Based Sofia \n\n\n\n\n\t\t\t\t\t\t \t\t\t\t\t\t \t\tThis role calls for exceptional client-facing communication skills, utmost...",07.05.21,5.0,#66c1ff,IT support Technician (Night Shift) Office Based Sofia,София; Заплата от 1800 до 2000 BGN (Нето),"Android, Windows, Active Directory"
3,5653146,https://www.jobs.bg/job/5653146,Съдърланд Глобъл Сървисиз България ЕООД,Разглеждания: 682,"Experte für Kundenlösungen\r\n \n\nInteressierst Du Dich für Handys und Smartphones, Zubehör und Dienstleistungen drumherum? Bist Du ein Verkaufstalent? Dann ist dieses Angebot ge...",07.05.21,5.0,#66c1ff,Experte für Kundenlösungen,София; Възможност за работа от вкъщи,Немски
4,5663138,https://www.jobs.bg/job/5663138,ТЕК ЕКСПЪРТС ЕООД,Разглеждания: 202,Customer Support Specialist with English and German\nAre you passionate about customer service? Are you looking for an organization where your career can develop rapidly within a short time frame?...,07.05.21,5.0,#66c1ff,Customer Support Specialist with English and German,София,
5,5661618,https://www.jobs.bg/job/5661618,ТИМ БЪЛГАРИЯ ЕООД,Разглеждания: 601,"ПОМОЩНИК-ГОТВАЧИ \n\n\n\n\n\t\t\t\t\t\t \t\t\t\t\t\t \tПомощник-Готвачи,\n\nХОТЕЛ ""ТИШИНА РЕЗИДЕНС "" гр.Приморско\n\n\nтърси да назначи готвачи за своите заведения:\n\nГ...",07.05.21,5.0,#66c1ff,ПОМОЩНИК-ГОТВАЧИ,Приморско; Заплата от 1000 до 1200 BGN (Нето),
6,5659182,https://www.jobs.bg/job/5659182,Съдърланд Глобъл Сървисиз България ЕООД,Разглеждания: 626,"For our newest client in Sofia, one of the most positive and globally recognized manufacturers of confectionery, and food products, we are hiring Customer Support Consultant with German to provide...",07.05.21,5.0,#66c1ff,Customer Support Consultant with German – Day Shifts,София; Възможност за работа от вкъщи,"Английски, Немски"
7,5665130,https://www.jobs.bg/job/5665130,ДОСТАВКИ БГ ООД,Разглеждания: 2 438,"Доставчици с лични и служебни Автомобили, Скутер и Велосипед \n\n\n\n\n\t\t\t\t\t\t \t\t\t\t\t\t \tФирма ''Доставки БГ ООД'' търси да назначи доставчик на храна със слу...",07.05.21,5.0,#66c1ff,"Доставчици с лични и служебни Автомобили, Скутер и Велосипед",София; Заплата от 1200 до 1800 BGN (Нето),
8,5669897,https://www.jobs.bg/job/5669897,ГРАФИКСОФТ ООД,Разглеждания: 218,"Grafixoft has 23 years of experience in software engineering and we are most proud of the long-lasting partnerships we have established over the years. Our partners are global leaders, and they st...",07.05.21,5.0,#66c1ff,SaaS Product System Administrator,София,"AWS, Windows, Linux"
9,5648367,https://www.jobs.bg/job/5648367,Wingam Ltd,Разглеждания: 831,"COMPANY\nAt Wingam we take pride in providing our clients with in-depth expert knowledge and competence in our services.\r\nThrough our vast experience and guaranteed degree of excellence, we offe...",07.05.21,5.0,#66c1ff,Personal Assistant of the General Manager,София,


In [7]:
job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})

for i in range(0, 45, 3):
    text_info_elem = job_infos[i]
    job_logo_elem = job_infos[i+1]
    job_company_elem = job_infos[i+2]
   
print(job_company_elem.text.strip())

Астреа Рикрутмънт ЕООД


In [8]:
driver.get(position_url)
html = driver.execute_script("return document.documentElement.outerHTML")
position_soup = BeautifulSoup(html, 'html.parser')

In [9]:
#position_r = requests.get(position_url)
#position_soup = BeautifulSoup(position_r.text, 'html.parser')

position_views = position_soup.find('span', string = 'Разглеждания:').parent

In [10]:
position_views.text

'Разглеждания: 136'

In [17]:
conda list

# packages in environment at /Users/LyubomirKushev/opt/anaconda3:
#
# Name                    Version                   Build  Channel
_anaconda_depends         2020.07                  py38_0  
_ipyw_jlab_nb_ext_conf    0.1.0                    py38_0  
alabaster                 0.7.12             pyhd3eb1b0_0  
anaconda                  custom                   py38_1  
anaconda-client           1.7.2                    py38_0  
anaconda-navigator        2.0.1                    py38_0  
anaconda-project          0.9.1              pyhd3eb1b0_1  
anyio                     2.2.0            py38hecd8cb5_1  
applaunchservices         0.2.1                      py_0  
appnope                   0.1.2           py38hecd8cb5_1001  
appscript                 1.1.2            py38h9ed2024_0  
argh                      0.26.2                   py38_0  
argon2-cffi               20.1.0           py38h9ed2024_1  
asn1crypto                1.4.0                      py_0  
astroid                